# Hamilton Standard Propulsion Model

In the 1970s, NASA contracted Hamilton Standard to forecast into the future mid-80s to the 90s what they thought advanced propellers would look like.
The result is what we call “Hamilton Standard model” used in Aviary today.
The [Hamilton Standard Documentation](https://ntrs.nasa.gov/api/citations/19720010354/downloads/19720010354.pdf) is publicly available.
You can find the definitions, methodology, and Fortran code in the document.
In Aviary, we implement only one of the computation options: the code computes the corresponding thrust for a given horsepower.

Below is an XDSM diagram of Hamilton Standard model (assuming `Dynamic.Mission.INSTALLATION_LOSS_FACTOR` is computed):

![Hamilton Standard Diagram](images/hamilton_standard.png)

The inputs are grouped in three aspects:

Geometric inputs:

- Propeller diameter
- Activity factor per blade (range: 80 to 200, baseline: 150)
- Number of blades (range: 2 to 8)

Power inputs:

- Shaft power to propeller (hp)
- Installation loss factor (0 to 1)

Performance inputs:

- Operating altitude (ft)
- True airspeed (knots)
- Propeller tip speed (Usually < 800 ft/s)
- Integrated lift coefficient (range: 0.3 to 0.8, baseline: 0.5)

Some of the inputs are valid for limited ranges.
When using an odd number of blades, the Hamilton Standard model interpolates using the 2, 4, 6 and 8 blade data.
The corresponding outputs are:

Geometric outputs:

- Design blade pitch angle (at 0.75 Radius)

Power outputs:

- Installation loss factor
- Tip compressibility loss factor
- Power coefficient
- Thrust coefficient (rho=const, no losses)

Performance outputs:

- Flight Mach number
- Propeller tip Mach number
- Advance ratio
- Tip compressibility loss factor
- Thrust
- Propeller efficiency with compressibility losses
- Propeller efficiency with compressibility and installation losses

When shaft power is zero, propeller efficiencies are undefined. We set them as 0.0.

As shown in the above XDSM diagram, the model is an OpenMDAO group that is composed of four components and one subgroup:

- `USatmos`
- `PreHamiltonStandard`
- `HamiltonStandard`
- `InstallLoss`
- `PostHamiltonStandard`

In [ ]:
# Testing Cell
import openmdao.api as om
from aviary.subsystems.atmosphere.atmosphere import Atmosphere
from aviary.subsystems.propulsion.propeller.propeller_performance import PropellerPerformance, InstallLoss
import aviary.subsystems.propulsion.propeller.hamilton_standard as hs
from aviary.variable_info.options import get_option_defaults
import aviary.api as av

options = get_option_defaults()
options.set_val(av.Aircraft.Engine.COMPUTE_PROPELLER_INSTALLATION_LOSS, val=True, units='unitless')
options.set_val(av.Aircraft.Engine.NUM_PROPELLER_BLADES, val=4, units='unitless')
options.set_val(av.Aircraft.Engine.GENERATE_FLIGHT_IDLE, False)
options.set_val(av.Aircraft.Engine.DATA_FILE, 'models/engines/turboshaft_4465hp.deck')
options.set_val(av.Aircraft.Engine.USE_PROPELLER_MAP, val=False)

prob = om.Problem()
group = prob.model
for name in ('traj','cruise','rhs_all'):
    group = group.add_subsystem(name, om.Group())
var_names = [
    (av.Aircraft.Engine.PROPELLER_TIP_SPEED_MAX,0,{'units':'ft/s'}),
    # (av.Dynamic.Mission.PERCENT_ROTOR_RPM_CORRECTED,0,{'units':'unitless'}),
    (av.Aircraft.Engine.PROPELLER_ACTIVITY_FACTOR,0,{'units':'unitless'}),
    (av.Aircraft.Engine.PROPELLER_INTEGRATED_LIFT_COEFFICIENT,0,{'units':'unitless'}),
    ]
group.add_subsystem('ivc',om.IndepVarComp(var_names),promotes=['*'])

prob.model.add_subsystem(
    name='atmosphere',
    subsys=Atmosphere(num_nodes=1),
    promotes=['*']
)

pp = prob.model.add_subsystem(
    'pp',
    PropellerPerformance(aviary_options=options),
    promotes_inputs=['*'],
    promotes_outputs=["*"],
)
pp.set_input_defaults(av.Aircraft.Engine.PROPELLER_DIAMETER, 10, units="ft")
pp.set_input_defaults(av.Dynamic.Mission.MACH, .7, units="unitless")
# pp.set_input_defaults(av.Dynamic.Mission.TEMPERATURE, 650, units="degR")
pp.set_input_defaults(av.Dynamic.Mission.PROPELLER_TIP_SPEED, 800, units="ft/s")
pp.set_input_defaults(av.Dynamic.Mission.VELOCITY, 100, units="knot")
prob.setup()

subsyses = {
    'install_loss':InstallLoss,
    'pre_hamilton_standard':hs.PreHamiltonStandard,
    'hamilton_standard':hs.HamiltonStandard,
    'post_hamilton_standard':hs.PostHamiltonStandard,
}

for name, component in subsyses.items():
    subsys = pp._get_subsystem(name)
    if subsys is None:
        raise ValueError(f"couldn't find {name} in PropellerPerformance")
    if not isinstance(subsys, component):
        raise TypeError(f'PropellerPerformance component {name} is {type(subsys)}, but should be {component}')

The `atmosphere` component provides the flight conditions.
The flight conditions are passed to the `PreHamiltonStandard` component which computes the propeller tip Mach number, advance ratio, and power coefficient.
These values are then fed into the `HamiltonStandard` component.

`HamiltonStandard` is the core of the model.
Given the power coefficient (CP) and advance ratio (J), it finds the blade angle (BL) by a CP-BL chart by tracing the advance ratio.
Then with the blade angle, it finds the thrust coefficient (CT) using its CT-BL chart by tracing advance ratio again.
This algorithm is shown in the below pair of charts.
The CP → BL → CT chart matching algorithm is based on baseline data.
If the user-inputted values are not in the valid region, it will first convert them to those baseline parameters by a sequence of interpolations to do the necessary corrections.
The newly converted parameters are called “effective parameters” (e.g., CPE and CTE).
The outputs are blade angle, thrust coefficient and tip compressibility loss factor.

![CP and CT matching](images/CPE_CTE_matching.png)

Finally, the thrust is computed in the `PostHamiltonStandard` component based on thrust coefficient and tip compressibility loss factor.

The Hamilton Standard model uses wind tunnel test data from uninstalled propellers.
When a nacelle is mounted behind the propeller, an installation loss factor is introduced.
The installation loss factor can be given by the user or computed.
If it is computed, we need another group of components as shown below:

![Installation Loss Factor](images/installation_loss_factor.png)

This diagram is represented by `InstallLoss` group in the first diagram.
Nacelle diameter is needed when installation loss factor is computed.
We use the average nacelle diameter.

The newly added aviary options and variables are:

In [ ]:
# Testing Cell
from aviary.api import Aircraft, Dynamic
from aviary.subsystems.propulsion.turboprop_model import TurbopropModel

In [ ]:
Aircraft.Engine.PROPELLER_DIAMETER
Aircraft.Engine.PROPELLER_INTEGRATED_LIFT_COEFFICIENT
Aircraft.Engine.PROPELLER_ACTIVITY_FACTOR
Aircraft.Engine.NUM_PROPELLER_BLADES
Aircraft.Engine.COMPUTE_PROPELLER_INSTALLATION_LOSS
Dynamic.Mission.PROPELLER_TIP_SPEED
Dynamic.Mission.SHAFT_POWER

To build a turboprop engine that uses the Hamilton Standard propeller model we use a `TurboPropModel` object with `propeller_model` set to `True` and `shaft_power_model` set to `False` (the default):

In [ ]:
engine = TurbopropModel(options=options, shaft_power_model=None, propeller_model=True)

Some inputs are options:

In [ ]:
options.set_val(Aircraft.Engine.PROPELLER_DIAMETER, 10, units='ft')
options.set_val(Aircraft.Engine.NUM_PROPELLER_BLADES, val=4, units='unitless')
options.set_val(Aircraft.Engine.COMPUTE_PROPELLER_INSTALLATION_LOSS, val=True, units='unitless')

We set the inputs like the following:

In [ ]:
prob.set_val(f'traj.cruise.rhs_all.{Aircraft.Engine.PROPELLER_TIP_SPEED_MAX}', 710., units='ft/s')
prob.set_val(f'traj.cruise.rhs_all.{Aircraft.Engine.PROPELLER_ACTIVITY_FACTOR}', 150., units='unitless')
prob.set_val(f'traj.cruise.rhs_all.{Aircraft.Engine.PROPELLER_INTEGRATED_LIFT_COEFFICIENT}', 0.5, units='unitless')

# Propeller Map Alternative

The Hamilton Standard model has limitations where it can be applied; for model aircraft design, it is possible that users may want to provide their own data tables. Two sample data sets are provided in `models/propellers` folder: `general_aviation.prop` and `PropFan.prop`. In both cases, they are in `.csv` format and are converted from `GASP` maps: `general_aviation.map` and `PropFan.map` (see [Command Line Tools](aviary_commands.ipynb) for details). The difference between these two samples is that the generatl aviation sample uses helical Mach numbers as input while the propfan sample uses the free stream Mach numbers. Helical Mach numbers appear higher, due to the inclusion of the rotational component of the tip velocity. In our example, they range from 0.7 to 0.95. To determine which mach type in a GASP map is used, please look at the first integer of the first line. If it is 1, it uses helical mach; if it is 2, it uses free stream mach. To determin which mach type is an Aviary propeller file is used, please look at the second item in the header. It is either `Helical_Mach` or `Mach`.

To use a propeller map, users can set `Aircraft.Engine.USE_PROPELLER_MAP` to `True` and provide the propeller map file path to `Aircraft.Engine.PROPELLER_DATA_FILE`. If helical Mach numbers are in the propeller map file, then an `OutMachs` component is added to convert helical Mach numbers to flight Mach numbers (`Dynamic.Mission.MACH`).

In the Hamilton Standard models, the thrust coefficients do not take compressibility into account. Therefore, propeller tip compressibility loss factor has to be computed and will be used to compute thrust. If a propeller map is used, the compressibility effects should be included in the data provided. Therefore, this factor is assumed to be 1.0 and is supplied to post Hamilton Standard component. Other outputs are computed using the same formulas.